# 00 - Ingesta y Optimización de Datos del CIC-IDS-2017

**Objetivo:** Este notebook se encarga de leer los 8 archivos CSV originales del dataset, unirlos, realizar una limpieza inicial y guardarlos en un único archivo Parquet optimizado para su uso en análisis posteriores.

In [1]:
import pandas as pd
import numpy as np
import os
import glob

## 1. Carga y Unión de Archivos CSV

En esta sección, se localizan todos los archivos **.csv** dentro de la carpeta de datos crudos. Cada archivo se carga en un DataFrame de Pandas y luego se concatenan todos en un único DataFrame para facilitar su manejo.

In [2]:
# Cargar y unir los archivos CSV
# Definir la ruta donde están los archivos CSV
path_to_csvs = r'../data/raw/DatasetCICIDS2017/MachineLearningCVE/'  # Cambia esta ruta según la ubicación de tus archivos CSV

# Se utiliza glob para encontrar todos los archivos CSV en el directorio
csv_files = glob.glob(os.path.join(path_to_csvs, "*.csv")) # Lista de todos los archivos CSV en el directorio

# Leer cada csv y guardarlo en una lista de dataframes
df_list = [] # Lista para almacenar los dataframes
for file_name in csv_files: # Iterar sobre cada archivo CSV
    df = pd.read_csv(file_name) # Leer el archivo CSV
    df_list.append(df) # Añadir el dataframe a la lista

# Concatenar todos los dataframes en uno solo
data = pd.concat(df_list, ignore_index = True) # Concatenar todos los dataframes en uno solo
print(f"Dimensiones del dataframe completo: {data.shape}") # Imprimir las dimensiones del dataframe completo

Dimensiones del dataframe completo: (2830743, 79)


## 2. Limpieza Inicial

Esta fase prepara el DataFrame para el análisis:
1.  **Limpieza de Nombres:** Se eliminan los espacios en blanco de los nombres de las columnas.
2.  **Manejo de Nulos:** Se reemplazan los valores infinitos (artefactos comunes en este dataset) por **NaN** y luego se eliminan las filas correspondientes.

In [3]:
# Limpieza y optimización inicial
data.columns = data.columns.str.strip() # Eliminar espacios en los nombres de las columnas

# El dataset CIC-IDS-2017 puede contener valores infinitos 
# Los reemplazamos con NaN (Not a Number) y luego eliminamos esas filas
data.replace([np.inf, -np.inf], np.nan, inplace = True)
data.dropna(inplace = True)
print(f"Nuevas dimensiones del DataFrame: {data.shape}")
data.isnull().sum().sum() # Verificar que no haya valores nulos

Nuevas dimensiones del DataFrame: (2827876, 79)


np.int64(0)

## 3. Optimización Inicial y Guardado en Formato Parquet

Finalmente, se aplica una función para reducir drásticamente el uso de memoria del DataFrame limpio, convirtiendo las columnas numéricas al tipo de dato más pequeño posible (**downcasting**) y las columnas de texto a un tipo **category** eficiente. El DataFrame optimizado se guarda en formato **Parquet**. Este formato es columnar, comprimido y mucho más rápido de leer que el CSV, lo que agilizará todos los análisis futuros.

In [4]:
# Función para reducir el uso de memoria del DataFrame
def reduce_memory_usage(data):
    start_mem = data.memory_usage().sum() / 1024**2 # Memoria inicial en MB
    print(f"Memoria usada por el DataFrame: {start_mem:.2f} MB") 

    for col in data.columns: # Iterar sobre cada columna del DataFrame
        col_type = data[col].dtype # Tipo de dato de la columna

        if col_type != object and col_type.name != "category": # Si la columna no es de tipo object (cadena de texto)
            c_min = data[col].min() # Valor mínimo de la columna
            c_max = data[col].max() # Valor máximo de la columna

            if str(col_type)[:3] == "int": # Si la columna es de tipo entero
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max: # Verificar si cabe en int8
                    data[col] = data[col].astype(np.int8) # Convertir a int8
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max: # Verificar si cabe en int16
                    data[col] = data[col].astype(np.int16) # Convertir a int16
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max: # Verificar si cabe en int32
                    data[col] = data[col].astype(np.int32) # Convertir a int32
                elif c_min >= np.iinfo(np.int64).min and c_max <= np.iinfo(np.int64).max: # Verificar si cabe en int64
                    data[col] = data[col].astype(np.int64) # Convertir a int64
            else:
                if c_min >= np.finfo(np.float16).min and c_max <= np.finfo(np.float16).max: # Verificar si cabe en float16
                    data[col] = data[col].astype(np.float16) # Convertir a float16
                elif c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max: # Verificar si cabe en float32
                    data[col] = data[col].astype(np.float32) # Convertir a float32
                else:
                    data[col] = data[col].astype(np.float64) # Convertir a float64
        else:
            data[col] = data[col].astype("category") # Convertir columnas de tipo object a category

    end_mem = data.memory_usage().sum() / 1024**2 # Memoria final en MB
    print(f"Memoria usada después de la optimización: {end_mem:.2f} MB") 
    print(f"Reducción de memoria: {100 * (start_mem - end_mem) / start_mem:.1f}%")
    return data 

data = reduce_memory_usage(data) # Aplicar la función de reducción de memoria
data.to_parquet(r"../data/processed/cic_ids_2017_optimized.parquet") # Guardar el DataFrame optimizado en formato Parquet

Memoria usada por el DataFrame: 1726.00 MB
Memoria usada después de la optimización: 655.34 MB
Reducción de memoria: 62.0%
